In [69]:
# Import required libraries
import os
import tensorflow as tf
import tensorflow_io as tfio
from matplotlib import pyplot as plt

In [19]:
# Dataset file path
DirPath = os.path.join('Dataset','Forest_Recordings')
file_name = 'recording_00.mp3'

file_path = os.path.join(DirPath, file_name)

In [63]:
# Function to load and resample audio files
def load_n_resample(file_path):
    sample_rate_in = 48000
    sample_rate_out = 16000

    file_data = tf.io.read_file(file_path)
    audio = tfio.audio.decode_mp3(file_data)

    audio = tf.math.reduce_sum(audio, axis=1)/2

    resampled_audio = tfio.audio.resample(audio, sample_rate_in, sample_rate_out)
    return resampled_audio

In [64]:
# Function to split large audio file into a series 3 sec clips
def slice_n_convrt_to_timeseries(audio):
    return tf.keras.preprocessing.timeseries_dataset_from_array(audio, None, 48000, 48000, 1, None)

In [65]:
# Function to convert audio files into spectrograms
def preprocess_audio(sample):
    audio_file = sample[:48000]                                                      # Select only first 3 secs of clip
    padding_length = 48000 - tf.shape(audio_file)[0]
    padding = tf.zeros([padding_length], dtype=tf.float32)                           # Create padding
    audio_file = tf.concat([audio_file, padding], 0)

    spectrogram = tf.signal.stft(audio_file, frame_length = 320 , frame_step = 32)   # Generates spectrogram using short time fourier transform
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis = 2) 

    spectrogram.set_shape([1491, 257, 1])                                            # Set the shape explicitly
    return spectrogram

In [87]:
# Load and process audio file
audio_file = load_n_resample(file_path)
sliced_dataset = slice_n_convrt_to_timeseries(audio_file)

sliced_dataset = sliced_dataset.map(preprocess_audio)
sliced_dataset = sliced_dataset.batch(1)

In [88]:
# Load pretrained model
model = tf.keras.models.load_model('./Models/audio_classifier.keras')

In [92]:
# Make prediction with confidence set to 90%
prediction = [1 if model.predict(sample) > 0.9 else 0 for sample in sliced_dataset.as_numpy_iterator()]
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

2024-07-13 19:35:31.487797: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [93]:
print("Total Calls:",tf.math.reduce_sum(prediction))

Total Calls: tf.Tensor(10, shape=(), dtype=int32)
